# Z3 : un problème ? Pas de problème !

In [1]:
from z3 import *  # la ligne qui ajoute 10 points de style

## Avant-propos

z3 est un **résolveur de contraintes** créé par Microsoft Research

# Le lycéen fainéant

Un lycéen qui n'en peut plus des Maths mais qui a des notions de Python décide de scripter la résolution de son exercice.

L'énoncé est le suivant :
"*Trouver des valeurs de a, b et c validant l'expression a + 2b + 3c = a + c tels que a != b != c a,b,c étant entiers*"

Le lycéen commence donc par **définir ses variables** avec z3

In [2]:
a = Int('a')  # définit une variable entière de valeur inconnue qui sera nommée comme 'a'
b = Int('b')
c = Int('c')

solver = Solver()  # déclare une instance du solveur de contraintes

Il explicite ensuite les **contraintes** de l'exercices

In [3]:
# a != b != c
solver.add(Distinct(a, b, c))
solver.add(a + 2 * b + 3 * c == a + c)

Il résout son exercice

In [4]:
solver.check()

sat

*sat* signifie que les contraintes ont été satisfaites, autrement dit que le solveur a trouvé des solutions aux contraintes !

In [5]:
solver.model()

[b = -1, a = -2, c = 1]

```-2 != -1 != 1```
et
```-2 + -1 * 3 + 1 * 3 = -2 + 1```
<=>
```-2 = -2```
l'exercice est résolu !

# Logique : opérations binaires

Ok maintenant le lycéen va jouer à Minecraft. Il va sur un serveur de mini-jeux et joue à un jeu de redstone. 
Pour ceux qui ne voient pas du tout ce qu'est la redstone, imaginez un circuit composés d'entrées : les leviers (ON => 1, OFF => 0) et des portes logiques (AND, OR, XOR).

Il a devant lui le circuit que l'on peut schématiser comme suit :

```
Levier 1 ===========
                   |= AND =
Levier 2 ===========      |= XOR =====
                          |          |
Levier 3 ==================          |== AND==>>
                                     |
Levier 4 =============================

"=" et "|" représentent le circuit
">>" représente la sortie
```

Le but du challenge, c'est d'activer les bons leviers pour qu'en sortie, on obtienne 1.
Ayant un terrible enthousiasme pour z3, il décide d'utiliser ce framework pour résoudre l'épreuve au lieu de réfléchir 1 sec.

In [6]:
# on définit les leviers (sachant que True == 1 == levier activé)
lev1 = Bool('lev1')
lev2 = Bool('lev2')
lev3 = Bool('lev3')
lev4 = Bool('lev4')

# on instancie notre solveur
solver_minecraft = Solver()

In [7]:
# on définit la logique de notre circuit
op1 = And(lev1, lev2)
op2 = Xor(op1, lev3)
op3 = And(op2, lev4)

solver_minecraft.add(op3)
# Remarque : solver_minecraft.add(op3) == solver_minecraft.add(op3 == True)

In [8]:
solver_minecraft.check()
solver_minecraft.model()

[lev1 = False, lev2 = False, lev3 = True, lev4 = True]

Une solution à ce problème est donc
```
    OFF  ==========
                   |= AND =
    OFF  ==========       |= XOR =====
                          |          |
     ON  ==================          |== AND==>> 1
                                     |
     ON  =============================
```
Si vous voulez voir quelque chose dans le même genre mais bien plus avancé :
[![LiveOverflowZ3](https://img.youtube.com/vi/nI8Q1bqT8QU/0.jpg)](https://youtu.be/nI8Q1bqT8QU)

# On s'énerve un petit coup

Bon maintenant on arrête l'histoire du lycéen et on se concentre sur de vrais problèmes.

### Sudoku

Rappel : le but du sudoku est de placer dans les cases des nombres, chaque nombre devant être unique sur sa ligne ou colonne.
Normalement il y a plusieurs zones dans un sudoku, ici on va simplement en considérer une.

In [9]:
def solve_sudoku(array: list) -> list or None:
    """ solves the sudoku game defines in map
    
    each element of the array will be transformed to z3 Int and
    constraints are added to specify that each line and colomn
    must not contain equal numbers
    
    :array is a list of sudoku game lines
    :returns the completed sudoku
    """
    solver = Solver()
    # double tableau ou chaque élément va être une instance Int()
    ints = []
    for line in range(len(array)):
        ints.append([])
        for element in range(len(array[line])):
            # on nomme chaque élément iCOLONNE_LIGNE
            ints[-1].append(Int(f"i{element}_{line}"))
            # tous les nombres doivent être compris entre 1 et le nombre d'éléments sur la ligne
            solver.add(And(ints[-1][-1] <= len(array[line]), ints[-1][-1] >= 1))
            # si l'élément actuel est initialisé, on ajoute la contrainte élément == valeur d'initialisation
            if array[line][element] != 0:
                solver.add(ints[-1][-1] == array[line][element])
        # chaque élément de la ligne qui vient d'être ajoutée doit être unique
        solver.add(Distinct(ints[-1]))
    # on créé des tableaux pour chaque colonne et on précise que chaque élément sur la colonne doit être unique
    for column in range(len(array)):
        check_colomns = []
        for line in range(len(ints)):
            check_colomns.append(ints[line][column])
        solver.add(Distinct(check_colomns))
    # on solve les contraintes
    c = solver.check()
    if c == unsat:
        print("No solution found, are you sure the puzzle can be solve ?")
        return None
    return solver.model()


game = [
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 2]
]

res = solve_sudoku(game)
print(res)

[i0_4 = 1,
 i1_1 = 5,
 i2_0 = 5,
 i3_2 = 3,
 i0_3 = 5,
 i2_3 = 3,
 i1_4 = 3,
 i2_4 = 4,
 i0_2 = 2,
 i3_3 = 4,
 i4_1 = 3,
 i3_4 = 5,
 i4_3 = 1,
 i1_2 = 4,
 i0_0 = 3,
 i4_0 = 4,
 i3_0 = 2,
 i2_1 = 2,
 i4_2 = 5,
 i1_0 = 1,
 i3_1 = 1,
 i1_3 = 2,
 i0_1 = 4,
 i4_4 = 2,
 i2_2 = 1]


Le résultat final est :
```
    5 4 3 2 1
    1 2 5 4 3
    2 3 1 5 4
    4 1 2 3 5
    3 5 4 1 2
```

### Le problèmes des nqueens

L'exercice des nqueens est un peu le hello world de l'algorithmie. Le but est très simple : on prend un échiquier  de n*n cases et le but est d'y placer le maximum de reines possibles sans qu'elles puissent s'attaquer. Pour rappel une reine peut attaquer sur toute une ligne, colonne ou diagonale.

On part donc du constat qu'il est impossible d'avoir deux reines sur la même ligne.

In [152]:
def print_chess(size: int, m: list) -> None:
    """displays solved chessboard
    """
    res = sorted([[d, m[d]] for d in m], key = lambda x: str(x[0]))
    j = 0
    print(str(res).count("1]"))
    for i in range(len(res)):
        if j < size - 1:
            print("O" if (res[i][1].as_long() == 0) else chr(0x2588), end="")
        if j == size - 1:
            print("O" if (res[i][1].as_long() == 0) else chr(0x2588))
            j = 0
        else:
            print(" ", end="")
            j += 1
    print()
    

queens = []
# n est le nombre de lignes et de colonnes
n = 8
solver = Solver()
for line in range(n):
    queens.append([])
    for colomn in range(n):
        queens[line].append(Int(f"q{line}_{colomn}"))
        solver.add(Or(queens[line][-1] == 1, queens[line][-1] == 0))
    solver.add(Or(sum(queens[-1]) == 1, sum(queens[-1]) == 0))
    
save = []
for colomn_i in range(n):
    colomn = []
    for line in range(n):
        colomn.append(queens[line][colomn_i])
        save.append(queens[line][colomn_i])
    solver.add(Or(sum(colomn) == 1, sum(colomn) == 0))
# there must be n queens
solver.add(sum(save) == n)
col = 0
lin = 0
for i in range(n):
    diag = []
    diag2 = []
    diag3 = []
    diag4 = []
    lin_down = i
    col_down = 0
    lin_up = n - 1
    col_up = i
    while col_down < n and lin_down < n:
        diag.append(queens[lin_down][col_down])  # i-down 0-right     ligne gauche vers bas
        diag2.append(queens[col_down][lin_down])  # 0-down i-right    ligne haut vers droite
        diag3.append(queens[lin_up][lin_down])  # n-1-up i-right      ligne bas vers droite
        diag4.append(queens[n - 1 - lin_down][col_down])  # i-up 0-right
        col_down += 1
        lin_down += 1
        lin_up -= 1
        col_up -= 1
    solver.add(Or(sum(diag) == 1, sum(diag) == 0))
    solver.add(Or(sum(diag2) == 1, sum(diag2) == 0))
    solver.add(Or(sum(diag3) == 1, sum(diag3) == 0))
    solver.add(Or(sum(diag4) == 1, sum(diag4) == 0))

c = solver.check()
print(c)
if c == sat:
    print_chess(n, solver.model())

sat
8
O O O O O █ O O
O O O █ O O O O
O O O O O O █ O
█ O O O O O O O
O O █ O O O O O
O O O O █ O O O
O █ O O O O O O
O O O O O O O █

